In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import operator

def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def test_eq(a,b): test(a,b,operator.eq,'==')

from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
def test_near(a,b): test(a,b,near)

In [3]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, mean, std_dev):
    return (x-mean)/std_dev
    

In [4]:
x_train, y_train, x_valid, y_valid = get_data()


#### normalize the train & valida input data.
see we use training values for valid data normalization

In [5]:
x_train_mean, x_train_std = x_train.mean(), x_train.std()
x_train_mean, x_train_std 

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, x_train_mean, x_train_std)
x_valid = normalize(x_valid, x_train_mean, x_train_std)

#### see diff in mean & std values

In [7]:
x_train_mean, x_train_std = x_train.mean(), x_train.std()
x_train_mean, x_train_std 

(tensor(3.0614e-05), tensor(1.))

In [8]:
def test_near_zero(a, tol=1e-3):
    assert a.abs()<tol, f"Near zero: {a}"

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

In [10]:
n, m = x_train.shape  
c = y_train.max()  + 1 # 1 extra class for the input which we cannot classify
n, m, c

(50000, 784, tensor(10))

### ARCHITECTURE

#### linear, relu, kaiming init & model

 we will first see the effect of random initialization vs [Kaiming initialization 2.2](https://arxiv.org/abs/1502.01852)

In [11]:
nh = 50 # hidden layers
# randn: Returns a tensor filled with random numbers from a normal distribution with mean `0` and variance `1`
w1 = torch.randn(m, nh)/math.sqrt(m)
w2 = torch.randn(nh, 1)/math.sqrt(m)
b1 = torch.zeros(nh)
b2 = torch.zeros(1)


In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()- 1/math.sqrt(m))

In [13]:
x_valid.mean(), x_valid.std() # close to 0 & 1

(tensor(-0.0058), tensor(0.9924))

In [14]:
def lin(x, w, b):
    # linear layer
    return x@w + b

t = lin(x_valid, w1, b1)
t.mean(), t.std()

(tensor(-0.0867), tensor(0.9584))

In [15]:
def relu(x):
    return x.clamp_min(0.)

t1 = relu(t) # t from lin(..)
t1.mean(), t1.std()

(tensor(0.3353), tensor(0.5328))

In [16]:
def kaiming_init(m, nh):
    return torch.randn(m, nh) * math.sqrt(2./m)

In [17]:
w1 = kaiming_init(m , nh)
t = relu(lin(x_valid, w1, b1))
w1.mean(), w1.std(), t.mean(), t.std()

(tensor(-0.0003), tensor(0.0507), tensor(0.5883), tensor(0.8508))

##### we can compare this kaiming_init with Pytorch kaiminig init

In [18]:
from torch.nn import init

In [19]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode="fan_out") 
#Choosing `fan_in` preserves the magnitude of the variance of the weights in the
#forward pass. Choosing `fan_out` preserves the magnitudes in the backwards pass.
t = relu(lin(x_valid, w1, b1))

In [20]:
w1.mean(), w1.std(), t.mean(), t.std()

(tensor(1.1033e-05), tensor(0.0507), tensor(0.5025), tensor(0.7761))

#### our relu is having mean > 0.5. Let's clip at 0.5 mercilessly.

In [21]:
def relu(x):
    return x.clamp_min(0.) - 0.5

In [22]:
w1 = kaiming_init(m, nh)
t1 = relu(lin(x_valid, w1, b1))
w1.mean(), w1.std(), t.mean(), t.std()

(tensor(-0.0002), tensor(0.0504), tensor(0.5025), tensor(0.7761))

### Our First model fun

In [23]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [24]:
# w2 = kaiming_init(nh, 1)

In [25]:
%timeit -n 10 _= model(x_valid)

8.45 ms ± 78.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

### Our First Loss Fun Mean Sq Error (MSE). 
to keep this simple

In [27]:
# squeeze will remove ,1] ; unsqueeze does the opposite
model(x_valid).shape, model(x_valid).squeeze(-1).shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [28]:
def mse(output, targ):
    return (output.squeeze(-1) - targ).pow(2).mean()

In [29]:
y_train, y_valid = y_train.float(), y_valid.float()

#### predict the output

In [30]:
preds = model(x_train)
x_train.shape, preds.shape, y_train.shape
# 50000 samples of 784 col (28 * 28 pic)

(torch.Size([50000, 784]), torch.Size([50000, 1]), torch.Size([50000]))

In [31]:
mse(preds, y_train)

tensor(29.1758)

### We will calculate the gradienct (i.e slope by differentiation) for backward pass
we will store gradient at `g` attribute

In [32]:
def mse_grad(inp, targ):
    # grad of loss wrt o/p of prev layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0] # this is gradient ;)

In [33]:
def relu_grad(inp, out):
    # grad of relu wrt input activations
    inp.g = (inp>0).float() * out.g

In [34]:
def lin_grad(inp, out, w, b):
    # grad of matrix mul wrt input:
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [35]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [36]:
forward_and_backward(x_train, y_train)

var w1, w2 b1 b2 are available because we declared it earlier

In [37]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [38]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

### Let's test if your calculation are correct and matches with Pytorch

In [39]:
xt2 = x_train.clone().requires_grad_(True) # true -> train it with grad calculations
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [40]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # no loss calc
    return mse(out, targ)

In [41]:
loss = forward(xt2, y_train)
loss.backward()

verify our calculations

In [42]:
w22.grad.shape, w2g.shape

(torch.Size([50, 1]), torch.Size([50, 1]))

In [43]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactoring
Lets convert fun into classes.

In [44]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [45]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # DONT TRY THIS AT HOME.
        # big matrix multiplication. There are better way to do this.
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1).sum(0)) 
        self.b.g = self.out.g.sum(0)

In [46]:
class Mse():
    def __init__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2 * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [50]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()
            

In [51]:
# Reset
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1, b1, w2, b2)

TypeError: __init__() missing 2 required positional arguments: 'inp' and 'targ'

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time model.backward()

verify our calculations

In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)
